In [ ]:
# Import Libraries
import os
import sys

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import wandb

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.collate import collate
from utils.load_data import load_word2vec
from utils.preprocess_str import str_to_tokens, tokenize
from utils.checkpoint import save_checkpoint
from models.core import DocumentDataset, TwoTowerModel, loss_fn


In [10]:
FREEZE_EMBEDDINGS = True
# Load embeddings
vocab,embeddings, word_to_idx = load_word2vec(embeddings_path='../models/word2vec/word2vec-gensim-text8-custom-preprocess.model')
embedding_layer = nn.Embedding.from_pretrained(embeddings, freeze=FREEZE_EMBEDDINGS)

EMBEDDING_DIM = embeddings.shape[1]
VOCAB_SIZE = len(vocab)
PROJECTION_DIM = 64

In [17]:
# # reload lib hard
# import importlib
# import utils
# importlib.reload(utils.preprocess_str)
# from utils.preprocess_str import str_to_tokens, tokenize

[nltk_data] Downloading package stopwords to /Users/x25bd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/x25bd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
# Load validation data
df = pd.read_parquet('./validation-tokenized.parquet')

# Only if we need to tokenize again
# df_original = pd.read_parquet('../data/validation.parquet')
# df_tokenized = tokenize(df_original, word_to_idx)

In [24]:
validationset = DocumentDataset(df)
validationloader = DataLoader(validationset, batch_size=1000, shuffle=False)

In [ ]:
model = TwoTowerModel(embedding_dim=EMBEDDING_DIM, 
                        projection_dim=PROJECTION_DIM, 
                        embedding_layer=embedding_layer, 
                        margin=MARGIN)